**Note**: Before running this notebook, make sure to run the `1. Baseline calculation.ipynb` notebook and output the updated `pkl` files in the `./interim_output/1. Baseline calculation/` needed to run this notebook

In [1]:
%load_ext autoreload
%autoreload 2

# <span style="color:#c6aa3d">0. Importing libraries and defining paths</span><a class="anchor" id="0-bullet"></a>

### 0.1 Importing libraries

In [2]:
# Importing python libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Setting the default max row and column display
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

In [4]:
# Path to src folder
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd())))
sys.path.append(SCRIPT_DIR)

# Import own scripts
import src.config as config
import src.custom_funcs as custom_funcs

C:\Users\Asus\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\Asus\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


### 0.2 Defining paths

In [5]:
# Input/Output Folder
notebook_output_path = "./output/2. RMI and Optimal Depth/"
baseline_interim_out_path = "./interim_output/1. Baseline calculation/"

# <span style="color:#c6aa3d">1. Reading interim files</span><a class="anchor" id="1-bullet"></a>

### 1.1 Importing model results

In [6]:
param_df_final = pd.read_pickle(baseline_interim_out_path + "param_df_final.pkl")
pred_df_final = pd.read_pickle(baseline_interim_out_path + "pred_df_final.pkl")
base_df_final = pd.read_pickle(baseline_interim_out_path + "base_df_final.pkl")

### 1.2 Importing model features

In [7]:
# Read model features data
df_feat_ohe = pd.read_pickle(baseline_interim_out_path + "features_df.pkl")

### 1.3 Importing SKU attributes mapping

In [8]:
# Read sku attributes mapping
df_sku_new_mapping = pd.read_pickle(baseline_interim_out_path + "sku_new_mapping.pkl")

### 1.4 Importing aggregated raw datasets

In [9]:
# Read aggregated raw modeling data
model_df_grouped = pd.read_pickle(baseline_interim_out_path + "model_df_grouped.pkl")
model_df_grouped = model_df_grouped.rename(columns={'week_start_date':'ds'})

# Read aggregated full raw data
full_df_grouped = pd.read_pickle(baseline_interim_out_path + "full_df_grouped.pkl")
full_df_grouped = full_df_grouped.rename(columns={'week_start_date':'ds'})

# <span style="color:#c6aa3d">2. Data processing</span><a class="anchor" id="2-bullet"></a>

### 2.1 Merging SKU-week prediction and baseline

In [10]:
# Merging pred and base df
pred_and_baseline = pred_df_final.copy()
pred_and_baseline = pd.merge(pred_and_baseline, 
                             base_df_final[["rfg", "ds", "yhat"]], 
                             how="inner", 
                             on=["rfg", "ds"])
pred_and_baseline = pred_and_baseline.rename(columns={"yhat_x": "yhat",
                                                      "yhat_y": "yhat_baseline"})

pred_and_baseline = pd.merge(pred_and_baseline,
                             df_feat_ohe[["rfg", "ds", "price", "base_price"]],
                             how="inner",
                             on=["rfg", "ds"])

assert pred_df_final.shape[0]==pred_and_baseline.shape[0], "[ERROR] Rows count has changed"

### 2.2 Checking grouping variable for analysis

In [11]:
# Check if group_OD_depth has one of the 2 values
# Banner should be one of the grouping variables
group_OD_depth = config.group_OD_depth

assert group_OD_depth == ['banner','BRAND_GROUP','CREDIT_CONSIGN'] or \
        group_OD_depth == ['banner','BRAND_GROUP','SUBCLASS','CREDIT_CONSIGN'], \
        "group_OD_depth variable should be either ['banner','BRAND_GROUP','CREDIT_CONSIGN'] or ['banner','BRAND_GROUP','SUBCLASS','CREDIT_CONSIGN']"

### 2.3 Filtering features data for final modeled SKUs

In [12]:
df_OD = df_feat_ohe.copy()

# Include only final shortlisted SKUs
df_OD = df_OD[df_OD['rfg'].isin(param_df_final['rfg'])]

### 2.4 Merging other information and calculating metrics

In [13]:
original_df_OD_count = df_OD.shape[0]

# Getting supplier funding metrics and excluding vat metrics
df_OD = df_OD.merge(model_df_grouped, on =['banner','sku_id_upd','ds'], how='inner')

# Getting sku attributes
df_OD = df_OD.merge(df_sku_new_mapping, on='sku_id_upd', how='inner')

# Getting baselines for each SKU-week
df_OD = df_OD.merge(pred_and_baseline[['rfg','ds','yhat_baseline']], on=['rfg','ds'], how='inner')

df_OD['TOTAL_PRODUCT_COST'] = df_OD['COST_OF_GOODS_SOLD'] \
                                .add(df_OD['PROMOTION_REBATE_AMT'], fill_value=0) \
                                .add(df_OD['SPECIAL_GP_AMT'], fill_value=0) \
                                .add(df_OD['DISCOUNT_SHARING_AMT'], fill_value=0)

df_OD['TOTAL_Supplier_Funding'] = df_OD['PROMOTION_REBATE_AMT'] \
                                    .add(df_OD['SPECIAL_GP_AMT'], fill_value=0) \
                                    .add(df_OD['DISCOUNT_SHARING_AMT'], fill_value=0)

df_OD['upd_total_disc'] = (df_OD['base_price'] - df_OD['price']) * df_OD['units']
df_OD['upd_GROSS_SALES'] = df_OD['base_price'] * df_OD['units']

assert df_OD.shape[0]==original_df_OD_count, "[ERROR] Rows count has changed"

# <span style="color:#c6aa3d">3. Calculating RMI</span><a class="anchor" id="3-bullet"></a>

### 3.1 Computing RMI for relevant timeframe

In [14]:
df_RMI = df_OD.copy()

# Filtering for specified timeframe
df_RMI = df_RMI[(df_RMI['ds'] >= config.OD_ds_start) & (df_RMI['ds'] <= config.OD_ds_end)]

In [15]:
df_RMI['baseline_sales'] = df_RMI['base_price'] * df_RMI['yhat_baseline']
df_RMI['baseline_cost'] = (df_RMI['TOTAL_PRODUCT_COST'] / df_RMI['units']) * df_RMI['yhat_baseline']
df_RMI['baseline_margin'] = df_RMI['baseline_sales'] - df_RMI['baseline_cost']

df_RMI['actual_margin'] = df_RMI['NET_SALES_AMT_INC_VAT'] - df_RMI['COST_OF_GOODS_SOLD']

df_RMI['margin_uplift'] = df_RMI['actual_margin'] - df_RMI['baseline_margin']

df_RMI = df_RMI.groupby(group_OD_depth).agg({'margin_uplift':'sum',
                                            'baseline_sales':'sum'}).reset_index()

df_RMI['RMI'] = df_RMI['margin_uplift']/df_RMI['baseline_sales']

### 3.2 Exporting results

In [16]:
df_RMI.to_csv(notebook_output_path + 'rmi_aggregated.csv', index=False)

# <span style="color:#c6aa3d">4. Optimal discount scenario planner input preparation</span><a class="anchor" id="4-bullet"></a>

### 4.1 Merging SKU parameter/coefficients

In [17]:
df_select_params_df_final = param_df_final[['rfg','discount',
                               'DISCOUNT_RULE_Line Discount', 'DISCOUNT_RULE_Mix&Match Amount',
                               'DISCOUNT_RULE_Mix&Match Quantity']]
df_select_params_df_final = df_select_params_df_final.add_prefix('coeff_').rename(columns={'coeff_rfg':'rfg'})

original_df_OD_count = df_OD.shape[0]

df_OD = df_OD.merge(df_select_params_df_final, on='rfg', how='inner')

df_OD['NS'] = df_OD['NET_SALES_AMT_INC_VAT']

assert df_OD.shape[0]==original_df_OD_count, "[ERROR] Rows count has changed"

### 4.2 Calculate weighted discount and mechanic coefficients

In [18]:
df_params_OD_coeff = df_OD.groupby(group_OD_depth+['sku_id_upd']).agg({
                                                    'coeff_discount':'mean',
                                                    'coeff_DISCOUNT_RULE_Line Discount':'mean',
                                                    'coeff_DISCOUNT_RULE_Mix&Match Amount':'mean',
                                                    'coeff_DISCOUNT_RULE_Mix&Match Quantity':'mean',    
                                                    'NS':'sum'}).reset_index()

for var in ['coeff_'+i for i in ['discount','DISCOUNT_RULE_Line Discount','DISCOUNT_RULE_Mix&Match Amount', 'DISCOUNT_RULE_Mix&Match Quantity']]:
    df_params_OD_coeff[var+'_times_NS'] = df_params_OD_coeff[var]* df_params_OD_coeff['NS']

df_params_OD_coeff = df_params_OD_coeff.groupby(group_OD_depth).agg({
                                                    'sku_id_upd':'count',
                                                    'coeff_discount_times_NS':'sum',
                                                    'coeff_DISCOUNT_RULE_Line Discount_times_NS':'sum',
                                                    'coeff_DISCOUNT_RULE_Mix&Match Amount_times_NS':'sum',
                                                    'coeff_DISCOUNT_RULE_Mix&Match Quantity_times_NS':'sum',
                                                    'NS':'sum'}).reset_index()

df_params_OD_coeff = df_params_OD_coeff.rename(columns={'sku_id_upd':'elasticity_sku_count'})

for var in ['coeff_'+i for i in ['discount','DISCOUNT_RULE_Line Discount','DISCOUNT_RULE_Mix&Match Amount', 'DISCOUNT_RULE_Mix&Match Quantity']]:
    df_params_OD_coeff[var+'_wtd_NS'] = df_params_OD_coeff[var+'_times_NS']/df_params_OD_coeff['NS']

df_params_OD_coeff = df_params_OD_coeff.loc[:,~(df_params_OD_coeff.columns.str.endswith('_times_NS'))]
df_params_OD_coeff = df_params_OD_coeff.drop(columns=['NS'])

### 4.3 Preparing Optimal discount scenario planner input for relevant timeframe

In [19]:
df_SP_OD = df_OD.copy()

# Filtering for timeframe
df_SP_OD = df_SP_OD[(df_SP_OD['ds'] >= config.OD_ds_start) & (df_SP_OD['ds'] <= config.OD_ds_end)]

In [20]:
df_SP_OD = df_SP_OD.groupby(group_OD_depth).agg({
                                        'sku_id_upd':'nunique',
                                        'TOTAL_Supplier_Funding':'sum',
                                        'TOTAL_PRODUCT_COST':'sum',
                                        'upd_GROSS_SALES':'sum',
                                        'upd_total_disc':'sum',
                                        'units':'sum',
                                        'yhat_baseline':'sum'}).reset_index()

df_SP_OD = df_SP_OD.rename(columns={'sku_id_upd':'modeled_sku_count'})

df_SP_OD['TPC_perunit'] = np.where(df_SP_OD['units']==0, 0, df_SP_OD['TOTAL_PRODUCT_COST']/df_SP_OD['units'])
df_SP_OD['base_price'] = np.where(df_SP_OD['units']==0, 0, df_SP_OD['upd_GROSS_SALES']/df_SP_OD['units'])
df_SP_OD['perc_supplier_funding'] = np.where(df_SP_OD['upd_total_disc']==0,0, df_SP_OD['TOTAL_Supplier_Funding']/df_SP_OD['upd_total_disc'])
df_SP_OD['perc_current_discount'] = df_SP_OD['upd_total_disc']/df_SP_OD['upd_GROSS_SALES']

# Creating excluding vat version of base price and % supplier funding
# Since we did base price flattening in baseline notebook, we can't calculate base price exc vat directly from GROSS_SALES_EXC_VAT
df_SP_OD['base_price'] = df_SP_OD['base_price']/(1 + config.VAT_perc)

# Supplier funding is represented as a proportion of discount
# Since discount is now including VAT, the denominator in perc_supplier_funding equation gets divided by (1+vat),
# effectively implying that %supplier funding gets multiplied by (1+vat)
df_SP_OD['perc_supplier_funding'] = df_SP_OD['perc_supplier_funding']*(1 + config.VAT_perc)

# Merge coefficients
df_SP_OD = df_SP_OD.merge(df_params_OD_coeff, on=group_OD_depth, how='inner')

### 4.4 Calculating net sales and profit coverage for extrapolation

In [21]:
df_OD_coverage = full_df_grouped.copy()
df_OD_coverage = df_OD_coverage.merge(df_sku_new_mapping, on='sku_id_upd', how='inner')

# Filtering for timeframe
df_OD_coverage = df_OD_coverage[(df_OD_coverage['ds']>=config.OD_ds_start) & (df_OD_coverage['ds']<=config.OD_ds_end)]

df_OD_coverage['Profit_exc_VAT'] = df_OD_coverage['NET_SALES_AMT_EXC_VAT'] - df_OD_coverage['COST_OF_GOODS_SOLD']

df_OD_coverage = df_OD_coverage.groupby(group_OD_depth).agg({'Profit_exc_VAT':'sum',
                                                            'NET_SALES_AMT_EXC_VAT':'sum'}).reset_index()

df_OD_coverage = df_OD_coverage.rename(columns={'Profit_exc_VAT':'Total_Profit_exc_VAT',
                                               'NET_SALES_AMT_EXC_VAT':'Total_NET_SALES_AMT_EXC_VAT'})

In [22]:
df_model_coverage = df_OD.copy()

# Filtering for timeframe
df_model_coverage = df_model_coverage[(df_model_coverage['ds']>=config.OD_ds_start) & (df_model_coverage['ds']<=config.OD_ds_end)]

df_model_coverage['Profit_exc_VAT'] = df_model_coverage['NET_SALES_AMT_EXC_VAT'] - df_model_coverage['COST_OF_GOODS_SOLD']

df_model_coverage = df_model_coverage.groupby(group_OD_depth).agg({'Profit_exc_VAT':'sum',
                                                            'NET_SALES_AMT_EXC_VAT':'sum'}).reset_index()

df_model_coverage = df_model_coverage.merge(df_OD_coverage, on=group_OD_depth, how='left')

df_model_coverage['Profit_Coverage_exc_VAT'] = df_model_coverage['Profit_exc_VAT']/df_model_coverage['Total_Profit_exc_VAT']
df_model_coverage['Net_Sales_Coverage_exc_VAT'] = df_model_coverage['NET_SALES_AMT_EXC_VAT']/df_model_coverage['Total_NET_SALES_AMT_EXC_VAT']

df_model_coverage = df_model_coverage[group_OD_depth+ ['Profit_exc_VAT','NET_SALES_AMT_EXC_VAT',
                                                'Total_Profit_exc_VAT','Total_NET_SALES_AMT_EXC_VAT',
                                                'Profit_Coverage_exc_VAT','Net_Sales_Coverage_exc_VAT']]

### 4.5 Merging profit and sales coverage to main input

In [23]:
original_SP_OD_rows = df_SP_OD.shape[0]

df_SP_OD = df_SP_OD.merge(df_model_coverage, on=group_OD_depth, how='inner')

assert original_SP_OD_rows == df_SP_OD.shape[0], "[ERROR] Rows count has changed"

### 4.6 Exporting optimal discount scenario output for populating scenario planner excel

In [24]:
# Creating a concatenated indicator for grouping fields
df_SP_OD['group_OD_depth'] = ''
for element in group_OD_depth:
    if element!='CREDIT_CONSIGN':
        df_SP_OD['group_OD_depth'] = df_SP_OD['group_OD_depth'] +'_'+ df_SP_OD[element]

df_SP_OD['group_OD_depth'] = df_SP_OD['group_OD_depth'].str.strip('_')

# Reordering columns- Moving group indicator to the beginning
group_OD_depth_column = df_SP_OD.pop('group_OD_depth')
df_SP_OD.insert(0, 'group_OD_depth', group_OD_depth_column)

In [25]:
df_SP_OD.to_csv(notebook_output_path + 'optimal_discount_scenario_input.csv', index=False)